In [80]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv

In [81]:
# Variáveis iniciais e load do .env
load_dotenv()

questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [82]:
# Load dos Modelos de Embedding e LLM
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-5-nano", max_tokens=5000)

In [83]:
# Carregar PDF
pdf_link = "os-sertoes.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

In [84]:
# Separar em chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [85]:
# Salvar no Vector DB (ChromaDB)
db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="./livro_index")

vectordb = Chroma(persist_directory="./livro_index", embedding_function=embeddings_model)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})	

In [86]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado

Contexto: {context}
    
Pergunta: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)
parser = StrOutputParser()
sequence = RunnableSequence(prompt | llm | parser)

In [87]:
# Função para fazer perguntas
def ask(question):
    context = retriever.invoke(question)
    response = sequence.invoke({"context": context, "question": question})
    return response

In [88]:
# Testar com uma pergunta
for index, question in enumerate(questions):

    resposta = ask(question)
    print(f"Número: {index + 1}")
    print(f"Pergunta: {question}")
    print(f"Resposta: {resposta}\n")

Número: 1
Pergunta: Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?
Resposta: Respondo com base no trecho fornecido e no conjunto da obra de Euclides da Cunha em Os Sertões. Observa-se que o “sertão” é apresentado como uma região de relevo áspero e imponente, cuja geografia — com planaltos, serras, rochas duras e rios que rasgam o terreno — é o que molda a vida de seus habitantes.

- Visão do ambiente natural: o sertão nordestino é descrito como uma paisagem de alto impacto físico, com rochas expostas, vertentes íngremes e uma topografia que se apresenta quase alpestre. A dureza do ambiente é enfatizada pela intensidade com que os rios deságuam e modelam os vales, bem como pela energia que o relevo impõe ao território.

- Influência sobre a vida dos habitantes: o ambiente impõe resistência, trabalho árduo e adaptação constante. A população precisa lidar com secas, variações climáticas e a necessidade de mobi